In [1]:
%load_ext blackcellmagic

In [2]:
from __future__ import annotations

import os
from pathlib import Path

Path().absolute()

PosixPath('/home/mix/Documents/SagasuSubs/providers/assrt')

In [3]:
SUBTITLE_PATH = Path().absolute() / 'downloads'/ 'subtitle'

assert SUBTITLE_PATH.is_dir()

In [4]:
import rarfile
import shutil


def _extract_rar(filename, extract_dir):
    try:
        rar = rarfile.RarFile(filename)
    except rarfile.Error as e:
        raise shutil.ReadError(f"{filename} is not a valid rar file") from e
    try:
        rar.extractall(extract_dir)
    finally:
        rar.close()


shutil.register_unpack_format("rar", [".rar"], _extract_rar)
shutil.get_unpack_formats()

[('bztar', ['.tar.bz2', '.tbz2'], "bzip2'ed tar-file"),
 ('gztar', ['.tar.gz', '.tgz'], "gzip'ed tar-file"),
 ('rar', ['.rar'], ''),
 ('tar', ['.tar'], 'uncompressed tar file'),
 ('xztar', ['.tar.xz', '.txz'], "xz'ed tar-file"),
 ('zip', ['.zip'], 'ZIP file')]

In [5]:
AVAILABLE_UNPACK_EXTS = {}
for name, exts, desc in shutil.get_unpack_formats():
    for ext in exts:
        AVAILABLE_UNPACK_EXTS[ext] = name

archive_files: list[Path] = []

for dir, _, files in os.walk(SUBTITLE_PATH):
    for file in files:
        path = SUBTITLE_PATH / dir / file
        name, ext = os.path.splitext(path.name)
        if ext in AVAILABLE_UNPACK_EXTS:
            archive_files.append(path)

archive_files[:3], len(archive_files)

([PosixPath('/home/mix/Documents/SagasuSubs/providers/assrt/downloads/subtitle/100023/downloads/_140530__PoRO___________________________________________________________________________________________________________________________________________________________________.zip'),
  PosixPath('/home/mix/Documents/SagasuSubs/providers/assrt/downloads/subtitle/100040/downloads/[Airota&VCB-Studio] Sora no Method [01-13+OVA] [CHT].rar'),
  PosixPath('/home/mix/Documents/SagasuSubs/providers/assrt/downloads/subtitle/100040/downloads/[FLsnow&SumiSora][Sora_no_Method][BDRIP][01-13+SP][GB+BIG5][ASS][SUB].rar')],
 11977)

In [ ]:
from tqdm.notebook import tqdm
from collections import defaultdict

progress = tqdm({*archive_files})

_finish_count = defaultdict(int)

for archive_file in progress:
    archive_file: Path
    name, ext = os.path.splitext(archive_file.name)
    extract_target = archive_file.parent / name
    relative_path = archive_file.relative_to(SUBTITLE_PATH)
    if extract_target.is_dir():
        shutil.rmtree(extract_target)
    progress.set_description(
        "succeed extracted: " + "\t".join(f"{k}={v}" for k, v in _finish_count.items())
    )
    try:
        shutil.unpack_archive(archive_file, extract_target)
    except Exception as e:
        print(f"--> {relative_path.name!r} unpack failed, read error: {e!r}")
    else:
        _finish_count[AVAILABLE_UNPACK_EXTS[ext]] += 1